In [0]:
#importing libraries
import numpy as np
import pandas as pd
#importing the natural language toolkit and downloading the important libraries
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [0]:
#installing docx2text library
pip install docx2txt

In [0]:
#importing docx2text and changing the docx file imported to text 
import docx2txt
k = docx2txt.process("text.docx")

In [0]:
#finding the number of characters in the text saved as in the name k
len(k)

23172

In [0]:
#tokenization basically refers to splitting up a larger body of text into smaller lines, words or even creating words for a non-English language.
#importing sent_tokenize function from nltk and sentence tokenizing the text 
from nltk.tokenize import sent_tokenize
corpus=sent_tokenize(k)
cor=corpus.copy()

In [0]:
#finding the number of sentences
len(cor)

131

In [0]:
#importing the re library and cleaning the text of any punctuation marks, numbers and eddy spaces
import re
corpuss=[]
for k in corpus:
  text = re.sub('[^a-zA-Z]', ' ', k)#allows just a-z and A-Z and replaces all other characters with space
  text = re.sub(r'\s+', ' ', text)#replaces multiple space sequences with a single space
  corpuss.append(text)
corpus=corpuss

In [0]:
#changing all the alphabetic characters to lower space
import string
corpuss=[s.lower() for s in corpus]
corpus=corpuss.copy()

In [0]:
#importing all the stopwords in the english language from the nltk package
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
def remove_stopwords(sen):                 ## Here we are removing all the stopwords form the sentence
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
cleaned = [remove_stopwords(r.split()) for r in corpuss]

In [0]:
len(stop_words)

179

In [0]:
#here we are defining a method to word tokenize
def Tokenization(corpus):        
    y=[]
    for i in range(0,len(corpus)):
      a=corpus[i].split(" ")
      y.append(a)
    return y

In [0]:
#defining a function to renove stop words and calling the Tokenization method to tokenize
def remove_stop_words(corpus):
 
    stop_words = nltk.corpus.stopwords.words('english')
  
    corpus_wo_stopwords=[]
    a=Tokenization(corpus)
    for i in a:
      x=[]
      for j in i:
        if j not in stop_words:
          x=x+[j]
      corpus_wo_stopwords+=[x]
        
    return corpus_wo_stopwords
#word tokenizing the raw text and removing stopwords from a copy of the raw text that we have kept untouched without doing sentence tokenization because these words will be used to find their word vectors    
PP_corpus = remove_stop_words(corpus)

In [0]:
#getting the unique words from the word tokenized list
def get_unique_words(PP_corpus):
    unique_words=set()
    for i in PP_corpus:
      for j in i:
          unique_words.add(j)
    return unique_words
  
unique_words = get_unique_words(PP_corpus)

In [0]:
#mapping each unique words in a dictionry against an integer
def mapping(unique_words):
    y1=unique_words
    ind=list(range(len(y1)))
    d = {k:v for k,v in zip(y1,ind)}
    return d
    
word2int = mapping(unique_words)

In [0]:
#getting the data using skip gram model.Skip gram model here is giving us the output as a list of tuples where each tuple has two elements. The first element is the centre word and the second element is the context word. It depends on thewindow length we have provided. 
import pandas as pd

def data_gen(PP_corpus, window_size):
   
    w=window_size
    y=PP_corpus
    li=[]
    k=0
    for i in range(len(y)):
      for j in range(len(y[i])):
        for h in range(-w,w+1):
          if(j+h>=0 and j+h<len(y[i])):
            if(not(j==j+h)):
              li+=[(y[i][j],y[i][j+h])]
    data = li   

    return data

    
data = data_gen(PP_corpus, 2)
df = pd.DataFrame(data, columns = ['input', 'label'])

In [0]:
data

[('artificial', 'intelligence'),
 ('artificial', 'ai'),
 ('intelligence', 'artificial'),
 ('intelligence', 'ai'),
 ('intelligence', 'science'),
 ('ai', 'artificial'),
 ('ai', 'intelligence'),
 ('ai', 'science'),
 ('ai', 'set'),
 ('science', 'intelligence'),
 ('science', 'ai'),
 ('science', 'set'),
 ('science', 'computational'),
 ('set', 'ai'),
 ('set', 'science'),
 ('set', 'computational'),
 ('set', 'technologies'),
 ('computational', 'science'),
 ('computational', 'set'),
 ('computational', 'technologies'),
 ('computational', 'inspired'),
 ('technologies', 'set'),
 ('technologies', 'computational'),
 ('technologies', 'inspired'),
 ('technologies', 'typically'),
 ('inspired', 'computational'),
 ('inspired', 'technologies'),
 ('inspired', 'typically'),
 ('inspired', 'operate'),
 ('typically', 'technologies'),
 ('typically', 'inspired'),
 ('typically', 'operate'),
 ('typically', 'quite'),
 ('operate', 'inspired'),
 ('operate', 'typically'),
 ('operate', 'quite'),
 ('operate', 'differentl

In [0]:
#defining a method for one-hot encoding
def to_one_hot_encoding(data_point_index):
   
    one_hot_encoding=np.zeros(len(unique_words))
    one_hot_encoding[data_point_index]=1
    return one_hot_encoding

In [0]:
#storing one hot encoded values of the data generated in X_train and Y_train where the index of the 1 is the value of the word in the dictionary
def one_hot_for_skip_gram(word2int, data):
    samples=len(data)
    vocab=len(word2int.keys())
    X=[]
    Y=[]
    for i in range(0,samples):
      X+=[to_one_hot_encoding(word2int[data[i][0]])]
      Y+=[to_one_hot_encoding(word2int[data[i][1]])]
    X=np.array(X)
    Y=np.array(Y)
    return X, Y
    
X_train,Y_train = one_hot_for_skip_gram(word2int, data)    

In [0]:
#importing  keras and all the required components for the model 
import keras

from keras.layers import Input,Dense
from keras.models import Model, Sequential
from keras import optimizers

In [0]:
#creating the model as we will find the word vectors of each word from the model. We define the model as such as that the length of the word vectors is the number of neurons in the last hidden layer.
def create_model():
    input_layer=Input(shape=(len(unique_words),))
    hidden_layer=Dense(11)(input_layer)
    output_layer=Dense(len(unique_words),activation='softmax')(hidden_layer)
    model=Model(input=[input_layer],output=[output_layer])
    model.compile(optimizer='Adam',loss='categorical_crossentropy')
    model.summary()
    return model

In [0]:
#creating the model and naming it as 'model'
model=create_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1031)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 11)                11352     
_________________________________________________________________
dense_4 (Dense)              (None, 1031)              12372     
Total params: 23,724
Trainable params: 23,724
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  """


In [0]:
#keeping a copy of the model
import copy
model_before = copy.copy(model)

In [0]:
#Model fitting is creating that simplified representation in a way that can generally be used successfully given new data.
model.fit(X_train, Y_train, epochs = 1000, batch_size = 200)

Epoch 1/1000
8418/8418 [==============================] - 1s 98us/step - loss: 6.9311
Epoch 2/1000
8418/8418 [==============================] - 1s 72us/step - loss: 6.9050
Epoch 3/1000
8418/8418 [==============================] - 1s 71us/step - loss: 6.8545
Epoch 4/1000
8418/8418 [==============================] - 1s 69us/step - loss: 6.7717
Epoch 5/1000
8418/8418 [==============================] - 1s 70us/step - loss: 6.6650
Epoch 6/1000
8418/8418 [==============================] - 1s 71us/step - loss: 6.5564
Epoch 7/1000
8418/8418 [==============================] - 1s 70us/step - loss: 6.4703
Epoch 8/1000
8418/8418 [==============================] - 1s 72us/step - loss: 6.4172
Epoch 9/1000
8418/8418 [==============================] - 1s 70us/step - loss: 6.3915
Epoch 10/1000
8418/8418 [==============================] - 1s 70us/step - loss: 6.3780
Epoch 11/1000
8418/8418 [==============================] - 1s 69us/step - loss: 6.3693
Epoch 12/1000
8418/8418 [===========================

In [0]:
#defining a method to get the enbeddings which is basically the list of the weights of connections of each neuron of the word output from the last hidden layer.
def get_embeddings(model, flag):
    embeddings=model.layers[flag+1].get_weights()[0]
    embeddings.reshape((len(unique_words),11))
    return embeddings

In [0]:
#gettig the embeddings by calling the method
embeddingst = get_embeddings(model, 0)

In [0]:
#saving the word embeddings and visualising it as a table
w2v_df = pd.DataFrame(embeddingst, columns = ['X1','X2','X3','X4','X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11'] )
w2v_df['word'] = unique_words
w2v_df = w2v_df[['word', 'X1', 'X2','X3','X4','X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11']]
w2v_df

,word,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11
0,,-0.030510,0.794130,-1.061649,-0.083442,-0.130550,-0.352269,0.181860,-0.089650,0.719598,0.729324,0.032105
1,high,-2.015454,0.884868,4.533864,0.341083,1.308593,0.774415,0.596743,-5.877708,-0.906213,1.543818,0.609791
2,protect,-0.487714,2.816028,-0.131617,-0.185775,-1.269962,0.442647,-1.234280,4.166347,1.736702,0.618867,0.986666
3,systems,0.292698,0.728003,1.879227,-0.000530,0.182456,-0.554824,0.675264,-0.770940,0.881297,0.236392,0.138573
4,closely,1.435934,0.594579,2.651418,-0.135553,-1.534516,0.055792,2.358037,0.954199,3.507608,0.667975,0.923649
5,fewer,-0.452418,1.659052,2.633829,-0.745144,3.918916,-2.328767,-1.142608,1.593076,-2.217718,0.309800,1.525806
6,within,-2.960756,-4.321366,-0.751060,0.267510,0.057799,-0.745357,0.974449,-2.248575,-3.089024,-1.595938,2.551947
7,collaboratively,1.194172,-0.755379,5.680677,-0.535311,-2.665900,-0.734639,1.185942,-0.941296,1.630548,2.094877,1.108514
8,california,2.510731,-0.982859,0.435630,3.191416,3.641332,-0.052642,-1.321754,-0.641195,0.131931,1.613738,-3.311498
9,science,-0.551139,-1.448377,-1.615890,0.856164,-1.522292,-0.753807,2.777269,0.226023,2.236964,-0.210313,-1.917423


In [0]:
#saving the word embeddings a dictionary of arrays where key is the word and the value is the wordvector in the form of an array
wordembeddings={}
for i in w2v_df['word']:
  vec=[]
  j=0
  vec.append(w2v_df['X1'][j])
  vec.append(w2v_df['X2'][j])
  vec.append(w2v_df['X3'][j])
  vec.append(w2v_df['X4'][j])
  vec.append(w2v_df['X5'][j])
  vec.append(w2v_df['X6'][j])
  vec.append(w2v_df['X7'][j])
  vec.append(w2v_df['X8'][j])
  vec.append(w2v_df['X9'][j])
  vec.append(w2v_df['X10'][j])
  vec.append(w2v_df['X11'][j])
  j+=1
  wordembeddings[i]= np.asarray(vec)

In [0]:
wordembeddings

{'': array([-0.03050997,  0.79413   , -1.061649  , -0.08344233, -0.13054961,
        -0.35226867,  0.18186016, -0.0896504 ,  0.7195977 ,  0.729324  ,
         0.03210484], dtype=float32),
 'high': array([-0.03050997,  0.79413   , -1.061649  , -0.08344233, -0.13054961,
        -0.35226867,  0.18186016, -0.0896504 ,  0.7195977 ,  0.729324  ,
         0.03210484], dtype=float32),
 'protect': array([-0.03050997,  0.79413   , -1.061649  , -0.08344233, -0.13054961,
        -0.35226867,  0.18186016, -0.0896504 ,  0.7195977 ,  0.729324  ,
         0.03210484], dtype=float32),
 'systems': array([-0.03050997,  0.79413   , -1.061649  , -0.08344233, -0.13054961,
        -0.35226867,  0.18186016, -0.0896504 ,  0.7195977 ,  0.729324  ,
         0.03210484], dtype=float32),
 'closely': array([-0.03050997,  0.79413   , -1.061649  , -0.08344233, -0.13054961,
        -0.35226867,  0.18186016, -0.0896504 ,  0.7195977 ,  0.729324  ,
         0.03210484], dtype=float32),
 'fewer': array([-0.03050997,  0.79

In [0]:
#finding the sentence vectors by adding the word vectors of the words in the sentence after cleaning them and dividing by the number of words in that sentence
sentence_vect=[]
for i in cleaned:
  if len(i)!=0:
    v = sum([wordembeddings.get(w, np.zeros((11,))) for w in i.split()])/(len(i.split())+0.000001)
  else:
    v=np.zeros((11,))
  sentence_vect.append(v)

TypeError: ignored

In [0]:
sentence_vect

[]

In [0]:
#defining the similarity matrix
sim_mat=np.zeros([len(cleaned),len(cleaned)])

In [0]:
#defing a method for finding out cosine similarity
def cossim(g,h):
  l=len(g)
  x,y,z=0,0,0
  for i in range(0,l):
    x=x+g[i]*h[i]
    y=y+g[i]**2
    z=z+h[i]**2
  x=x/(y*z)
  return x

In [0]:
#filling up the similarity matrix by finding out the similarity between sentences using cossim method
for i in range(0, len(cleaned)):
  for j in range(0, len(cleaned)):
    if i!=j:
      g=sentence_vect[i]
      h=sentence_vect[j]
      sim_mat[i][j]=cossim(g,h)

In [0]:
#similarity matrix
sim_mat

array([[0.        , 0.23189048, 0.23189049, ..., 0.23189046, 0.23189047,
        0.23189056],
       [0.23189048, 0.        , 0.2318905 , ..., 0.23189047, 0.23189049,
        0.23189058],
       [0.23189049, 0.2318905 , 0.        , ..., 0.23189049, 0.2318905 ,
        0.23189059],
       ...,
       [0.23189046, 0.23189047, 0.23189049, ..., 0.        , 0.23189047,
        0.23189055],
       [0.23189047, 0.23189049, 0.2318905 , ..., 0.23189047, 0.        ,
        0.23189057],
       [0.23189056, 0.23189058, 0.23189059, ..., 0.23189055, 0.23189057,
        0.        ]])

In [0]:
#finding the edge values ,that is by how much each sentence is similar to other sentences
edge=[]
for i in range(0,len(cleaned)):
  for j in range(i+1,len(cleaned)):
    dict={}
    dict['weight']=sim_mat[i][j]
    edge=edge+[(i,j,dict)]
edge=(edge)

In [0]:
edge

[(0, 1, {'weight': 0.23189047727328801}),
 (0, 2, {'weight': 0.23189049459967517}),
 (0, 3, {'weight': 0.23189047839817792}),
 (0, 4, {'weight': 0.23189045900924846}),
 (0, 5, {'weight': 0.23189049457186542}),
 (0, 6, {'weight': 0.23189048887947641}),
 (0, 7, {'weight': 0.23189048420026342}),
 (0, 8, {'weight': 0.23189047396457385}),
 (0, 9, {'weight': 0.23189049457186542}),
 (0, 10, {'weight': 0.23189047802240662}),
 (0, 11, {'weight': 0.23189047802240662}),
 (0, 12, {'weight': 0.23189043542913357}),
 (0, 13, {'weight': 0.23189047839817792}),
 (0, 14, {'weight': 0.23189044609665405}),
 (0, 15, {'weight': 0.23189047839817792}),
 (0, 16, {'weight': 0.23189048627175754}),
 (0, 17, {'weight': 0.23189048046625046}),
 (0, 18, {'weight': 0.23189047839817792}),
 (0, 19, {'weight': 0.23189049459967517}),
 (0, 20, {'weight': 0.23189046298773172}),
 (0, 21, {'weight': 0.23189048617699695}),
 (0, 22, {'weight': 0.23189042793582193}),
 (0, 23, {'weight': 0.2318904780183713}),
 (0, 24, {'weight': 0

In [0]:
#finding the score of each sentence and storing it in a dictionary. the score is the mean of the edge values for each sentence with every other sentence.
scores1={}
k=0
while(k<len(cor)):
  sum=0
  for i in edge:
    if i[0] == k:
      sum+=i[2]['weight']
    elif i[1] == k:
      sum+=i[2]['weight']
  sum=sum/len(cor)
  scores1[k]=sum 
  k+=1
  

In [0]:
scores1

{0: 0.23013373501068285,
 1: 0.23013374647045537,
 2: 0.23013376209717185,
 3: 0.2301337477239893,
 4: 0.23013373000072523,
 5: 0.23013376199542174,
 6: 0.23013375787651108,
 7: 0.23013375242593756,
 8: 0.23013374484630839,
 9: 0.23013376199542174,
 10: 0.23013374720822374,
 11: 0.23013374720822374,
 12: 0.23013370502204278,
 13: 0.2301337477239893,
 14: 0.23013371557042667,
 15: 0.2301337477239893,
 16: 0.23013375343864292,
 17: 0.23013374829170852,
 18: 0.2301337477239893,
 19: 0.23013376209717185,
 20: 0.23013373413138938,
 21: 0.2301337549372611,
 22: 0.23013370043006298,
 23: 0.23013374931217023,
 24: 0.23013376209717185,
 25: 0.23013371557042667,
 26: 0.2301336954220636,
 27: 0.2301337546528548,
 28: 0.23013374647045537,
 29: 0.23013374720822374,
 30: 0.23013375242593756,
 31: 0.23013374931217023,
 32: 0.2301337549372611,
 33: 0.23013374484630839,
 34: 0.23013375310021184,
 35: 0.23013375810996023,
 36: 0.23013375787651105,
 37: 0.23013375242593756,
 38: 0.23013376209717185,
 39:

In [0]:
ranked_sentences = sorted(((scores1[i],s) for i,s in enumerate(cor)), reverse=True)

In [0]:
ranked_sentences

[(0.32465956564658704, '10 AI policy, now and'),
 (0.32465952064885595, 'What’s next for AI research?'),
 (0.3246594612461253,
  'While impressive, these technologies are highly tailored to particular tasks.'),
 (0.3246594612461253,
  'AI is also changing how people interact with technology.'),
 (0.32465946045194916,
  'Great strides have also been made in machine translation among different languages, with more real-time person-to-person exchanges on the near horizon.'),
 (0.3246594604519491,
  'People’s future relationships with machines will become ever more nuanced, fluid, and personalized as AI systems learn to adapt to individual personalities and goals.'),
 (0.3246594604519491,
  'Once a mostly academic area of study, twenty-first century AI enables a constellation of mainstream technologies that are having a substantial impact on everyday lives.'),
 (0.3246594604519491,
  'On the other hand, if society approaches AI with a more open mind, the technologies emerging from the fiel

In [0]:
#sorting the sentences in the reverse manner for getting sentences with high scores and printing them.
for i in range(10):
  print(ranked_sentences[i][1])

10 AI policy, now and
What’s next for AI research?
While impressive, these technologies are highly tailored to particular tasks.
AI is also changing how people interact with technology.
Great strides have also been made in machine translation among different languages, with more real-time person-to-person exchanges on the near horizon.
People’s future relationships with machines will become ever more nuanced, fluid, and personalized as AI systems learn to adapt to individual personalities and goals.
Once a mostly academic area of study, twenty-first century AI enables a constellation of mainstream technologies that are having a substantial impact on everyday lives.
On the other hand, if society approaches AI with a more open mind, the technologies emerging from the field could profoundly transform society for the better in the coming decades.
Instead, increasingly useful applications of AI, with potentially profound positive impacts on our society and economy are likely to emerge betwe